In [71]:
import cv2
import numpy as np
from scipy import ndimage
import scipy
from scipy import ndimage
import imageio
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [72]:


start = time.time()
# outframe = hdmi_out.newframe()
#2D Sobel operator https://stackoverflow.com/questions/7185655/applying-the-sobel-filter-using-scipy
im = imageio.imread("cartoon.jpg")
# im = cv2.imread("cartoon.jpg")
im = im.astype('int32')
dx = ndimage.sobel(im, 0) # horizontal deriv
dy = ndimage.sobel(im, 1) # vertical deriv
mag = np.hypot(dx, dy) # magnitude
mag *= 255.0/np.max(mag) #normalize
cv2.imwrite("cartoon_mag.jpg", mag)

end = time.time()

print(end-start)

0.1527848243713379


In [73]:
start = time.time()

# https://medium.com/@nikatsanka/comparing-edge-detection-methods-638a2919476e

# sobel operator manually

# Apply gray scale
# gray_img = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
gray_img = np.round(0.299 * im[:, :, 0] +
                    0.587 * im[:, :, 1] +
                    0.114 * im[:, :, 2]).astype(np.uint8)

# Sobel Operator
h, w = gray_img.shape
# define filters
horizontal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])  # s2
vertical = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])  # s1

# define images with 0s
newhorizontalImage = np.zeros((h, w))
newverticalImage = np.zeros((h, w))
newgradientImage_sobel = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        newhorizontalImage[i - 1, j - 1] = abs(horizontalGrad)

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        newverticalImage[i - 1, j - 1] = abs(verticalGrad)

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_sobel[i - 1, j - 1] = mag

cv2.imwrite("cartoon_sobel.jpg", newgradientImage_sobel)

end = time.time()

print(end-start)

15.2987380027771


In [74]:

start = time.time()
#prewitt
horizontal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # s2
vertical = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])  # s1

# define images with 0s
newgradientImage_prewitt = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_prewitt[i - 1, j - 1] = mag


cv2.imwrite("cartoon_prewitt.jpg", newgradientImage_prewitt)

end = time.time()

print(end-start)

14.404906272888184


In [75]:
start = time.time()

#laplachian
horizontal = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]]) #positive laplachian
# horizontal = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]]) #negative laplachian
vertical = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]) 


# define images with 0s
newgradientImage_laplachian = np.zeros((h, w))

# offset by 1
for i in range(1, h - 1):
    for j in range(1, w - 1):
        horizontalGrad = (horizontal[0, 0] * gray_img[i - 1, j - 1]) + \
                         (horizontal[0, 1] * gray_img[i - 1, j]) + \
                         (horizontal[0, 2] * gray_img[i - 1, j + 1]) + \
                         (horizontal[1, 0] * gray_img[i, j - 1]) + \
                         (horizontal[1, 1] * gray_img[i, j]) + \
                         (horizontal[1, 2] * gray_img[i, j + 1]) + \
                         (horizontal[2, 0] * gray_img[i + 1, j - 1]) + \
                         (horizontal[2, 1] * gray_img[i + 1, j]) + \
                         (horizontal[2, 2] * gray_img[i + 1, j + 1])

        verticalGrad = (vertical[0, 0] * gray_img[i - 1, j - 1]) + \
                       (vertical[0, 1] * gray_img[i - 1, j]) + \
                       (vertical[0, 2] * gray_img[i - 1, j + 1]) + \
                       (vertical[1, 0] * gray_img[i, j - 1]) + \
                       (vertical[1, 1] * gray_img[i, j]) + \
                       (vertical[1, 2] * gray_img[i, j + 1]) + \
                       (vertical[2, 0] * gray_img[i + 1, j - 1]) + \
                       (vertical[2, 1] * gray_img[i + 1, j]) + \
                       (vertical[2, 2] * gray_img[i + 1, j + 1])

        # Edge Magnitude
        mag = np.sqrt(pow(horizontalGrad, 2.0) + pow(verticalGrad, 2.0))
        newgradientImage_laplachian[i - 1, j - 1] = mag

cv2.imwrite("cartoon_laplachian.jpg", newgradientImage_laplachian)

end = time.time()

print(end-start)

14.645830154418945


In [85]:
start = time.time()

#morphologic edge out https://stackoverflow.com/questions/61643039/improving-canny-edge-detection
img = cv2.imread("cartoon.jpg")

# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
threshold = cv2.threshold(gray_img, 180, 255, cv2.THRESH_BINARY)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilate = cv2.morphologyEx(threshold, cv2.MORPH_DILATE, kernel)

diff = cv2.absdiff(dilate, threshold)
# edges = 255-diff
        
# cv2.Canny(frame_webcam, 100, 110, edges=outframe)
cv2.imwrite("cartoon_edges.jpg", diff)

end = time.time()

print(end-start)

0.0292050838470459


In [83]:
start = time.time()

img2 = cv2.Canny(img, 100, 110)
# cannyEdges = 255-img2

cv2.imwrite('cartoon_canny.jpg', img2)

end = time.time()

print(end-start)

0.022442102432250977


In [95]:
cv2.imwrite("sobel-prewitt.jpg", newgradientImage_sobel-newgradientImage_prewitt)
cv2.imwrite("sobel-laplachian.jpg", newgradientImage_sobel-newgradientImage_laplachian)
cv2.imwrite("prewitt-laplachian.jpg", newgradientImage_prewitt-newgradientImage_laplachian)
cv2.imwrite("sobelmanual-sobelgiven.jpg", newgradientImage_sobel+mag)
cv2.imwrite("cartoon_gray.jpg", cv2.bitwise_not(img))


True

In [ ]:
# look into different gaussian filters x
# different edge out functions
# diferrent operator functions https://medium.com/@nikatsanka/comparing-edge-detection-methods-638a2919476e x